# Using the archive of `opsim` runs and summary metrics

## Introduction

The Rubin Observatory Survey Strategy Team is producing an extensive collection of survey strategy simulations (using `opsim`) and corresponding analysis (using `MAF`). Many of these are of interest for science collaborations, and are publicly available. One interface to this data is [interactive web page](http://astro-lsst-01.astro.washington.edu:8081/) with lists of simulation runs and links ot `opsim` configuration and output files (databases of scheduled visits with simulated data quality) and output of `MAF` including summary values and plots:

- http://astro-lsst-01.astro.washington.edu:8081/
    
A programmatic interface to this data is also sometimes helpful. The `rubin_sim.maf.archive` module provides such an interface.

## Basic concepts

Survey strategy executions and analysis are assigned names and collected into groups for easy management and reference, according to the following nomenclature:

| term | discussion |
|------|------------|
| run | A **run** is a single execution of `opsim`. Each run produces an SQLite database of visits with data describing each visit (e.g. the start time, filter used, simulated seeing, etc.). Each `run` has a canonical "run name". Examples include `baseline_nexp2_v1.7.1_10yrs`, `barebones_nexp2_v1.6_10yrs`, and `combo_dust_nexp2_v1.6_10yrs`.|
| family | The survey strategy team often produces collections of runs designed to explore a specific aspect of survey strategy. Different runs in a collection vary the aspect of survey strategy being studied, while keeping other aspects the same. Comparing runs that are part of the same collection or **family** therefore supports exploring the effects of varying a specific parameter or other scheduler feature. Other collections for which direct comparison might be useful can also sometimes be grouped into the same family. Examples include the `version_baselines` famile, which include simulations that have been used as "baselines" at different points in time; and `potential_schedulers`, which are runs that appeared to be good candidates using the `FBS_1.6` version of `opsim`. |
| summary metric | A **summary metric** is a single scalar representing some feature of an `opsim` run, generally one that indicates some aspect of the quality of the survey. Each `MAF` metric may produce any number of summary metrics, and each execution of MAF may construct an arbitrary number of summary metrics, depending on the MAF metric bundles executed. Each summary metric has a cannonical name derived from various elements of the metric bundle. |
| summary metric sets | Standard executions of MAF on opsim runs produce thousands of summary metrics, and users will only wish to inspect and compare limited subsets of these summary metrics at any given time. The survey strategy team has therefore pre-defined a collection of named sets of metrics, so that sets of metrics usefully examined as a group can be referenced together. Examples of summary metric sets include `srd` (which correspond to requirement in the Science Requirements Document), `Nvis` (which counts visits of different types), `tvs` (summary metrics of interest to the transients and variable stars working group), `descWFD` (summary metrics of interest to DESC analysis of the WFD survey), and more. | 

## Notebook preparation

The following is a development style aid; only uncomment if developing the notebook:

In [1]:
%load_ext lab_black
%load_ext pycodestyle_magic
%flake8_on --ignore E501,W505

Required imports:

In [2]:
import rubin_sim
from rubin_sim import maf

In [3]:
%load_ext autoreload
%autoreload 1
%aimport archive

## Run families

The "families" json file organizes `opsim` runs into "families," groups of runs that vary in a controlled way, and which are approprate for direct comparison with each other in order to understand the effects of varying specific parameters, or making specific alterations to survey strategy.

You can download a table of families, their descriptions, and definitions into a `pandas.DataFrame` thus:

In [4]:
families = archive.get_family_descriptions()
families

,version,reference,description,run,brief,filepath,url
family,,,,,,,
baseline,2,baseline_v2.0_10yrs,**baseline** : The v2.0 baseline simulation. T...,[baseline_v2.0_10yrs],[v2.0 baseline],[baseline/baseline_v2.0_10yrs.db],[http://epyc.astro.washington.edu/~lynnej/opsi...
retro,2,baseline_v2.0_10yrs,**retro** : The retro family provides two simu...,"[baseline_retrofoot_v2.0_10yrs, retro_baseline...","[v1.x footprint but new code capabilities, v1....","[retro/baseline_retrofoot_v2.0_10yrs.db, retro...",[http://epyc.astro.washington.edu/~lynnej/opsi...
rolling,2,baseline_v2.0_10yrs,**rolling** : The updated baseline contains a ...,"[rolling_ns2_rw0.5_v2.0_10yrs, rolling_ns3_rw0...","[Rolling half-sky (2 regions) 50%, Rolling thi...","[rolling/rolling_ns2_rw0.5_v2.0_10yrs.db, roll...",[http://epyc.astro.washington.edu/~lynnej/opsi...
rolling_bulge,2,baseline_v2.0_10yrs,**rolling_bulge** : The updated baseline conta...,"[rolling_bulge_ns2_rw0.5_v2.0_10yrs, rolling_b...","[Roll in the bulge with 2 regions, 50%, Roll i...",[rolling_bulge/rolling_bulge_ns2_rw0.5_v2.0_10...,[http://epyc.astro.washington.edu/~lynnej/opsi...
rolling_bulge_6,2,baseline_v2.0_10yrs,**rolling_bulge_6** : The updated baseline con...,[rolling_bulge_6_v2.0_10yrs],[Roll the bulge in 6 stripes (low-dust WFD in ...,[rolling_bulge_6/rolling_bulge_6_v2.0_10yrs.db],[http://epyc.astro.washington.edu/~lynnej/opsi...
rolling_six,2,baseline_v2.0_10yrs,**rolling_six** : The updated baseline contain...,"[six_rolling_ns6_rw0.5_v2.0_10yrs, six_rolling...","[Rolling in one-sixth sky (6 regions) 50%, Rol...",[rolling_six/six_rolling_ns6_rw0.5_v2.0_10yrs....,[http://epyc.astro.washington.edu/~lynnej/opsi...
rolling_early,2,baseline_v2.0_10yrs,**rolling_early** : The updated baseline conta...,[roll_early_v2.0_10yrs],[Start rolling year 1],[rolling_early/roll_early_v2.0_10yrs.db],[http://epyc.astro.washington.edu/~lynnej/opsi...
rolling_all_sky,2,baseline_v2.0_10yrs,**rolling_all_sky** : The updated baseline con...,[rolling_all_sky_ns2_rw0.9_v2.0_10yrs],[Roll on all parts of the sky],[rolling_all_sky/rolling_all_sky_ns2_rw0.9_v2....,[http://epyc.astro.washington.edu/~lynnej/opsi...
noroll,2,baseline_v2.0_10yrs,**noroll** : The updated baseline contains a 2...,[noroll_v2.0_10yrs],[v2.0 baseline with no rolling],[noroll/noroll_v2.0_10yrs.db],[http://epyc.astro.washington.edu/~lynnej/opsi...


By default, `get_family_descriptions` retrives the runs data from a json file at the URL provided in `archive.FAMILY_SOURCE`:

In [5]:
archive.FAMILY_SOURCE

'https://raw.githubusercontent.com/lsst-pst/survey_strategy/main/fbs_2.0/runs_v2.0.json'

If you wish to load runs from an alternate source, it can be specified with the `family_source` argument to `get_family_descriptions`.

You can use the loaded data to get a list of available families:

You can get more pleasantly formatted descriptions of the families using `archive.describe_families`:

In [6]:
archive.describe_families(families.loc[["baseline", "retro"]])

---
**baseline** : The v2.0 baseline simulation. This features a modified survey footprint with expanded dust-free area and WFD-level visits in the Galactic Bulge and Magellanic Clouds. Coverage of the Northern Ecliptic Spur, South Celestial Pole, and remainder of the Galactic Plane is maintained, at lower levels. Filter balance is modified in different areas of the sky. A 2-region rolling cadence is used in the dust-free WFD sky, to improve cadence.   
**version**: 2  
**runs**:  


,run,filepath
brief,,
v2.0 baseline,baseline_v2.0_10yrs,baseline/baseline_v2.0_10yrs.db


---
**retro** : The retro family provides two simulations to bridge the gap between previous sets of simulations and the updated baseline. One (retro_baseline) uses the current code base to re-run the previous baseline (i.e. this is baseline_nexp2_v1.7.1_10yrs, but run with rubin_sim) while the other (retrofoot) uses updated code settings but re-creates the baseline_nexp2_v1.7.1_10yrs footprint. These are primarily intended to help users understand potential changes in their metric results from the 1.x series of runs to the v2.0 runs, as well as the modification from sims_maf to rubin_sim.  
**version**: 2  
**runs**:  


,run,filepath
brief,,
v1.x footprint but new code capabilities,baseline_retrofoot_v2.0_10yrs,retro/baseline_retrofoot_v2.0_10yrs.db
v1.x baseline with rubin_sim (without new code capabilities),retro_baseline_v2.0_10yrs,retro/retro_baseline_v2.0_10yrs.db


## Getting a table of runs

You can download a `pandas.DataFrame` of runs with basic information on each using `get_runs`:

In [7]:
runs = archive.get_runs()
runs

,family,version,brief,filepath,url
run,,,,,
baseline_retrofoot_v2.0_10yrs,[retro],[2],v1.x footprint but new code capabilities,retro/baseline_retrofoot_v2.0_10yrs.db,http://epyc.astro.washington.edu/~lynnej/opsim...
baseline_v2.0_10yrs,[baseline],[2],v2.0 baseline,baseline/baseline_v2.0_10yrs.db,http://epyc.astro.washington.edu/~lynnej/opsim...
bluer_indx0_v2.0_10yrs,[bluer],[2],"More g - filter ratio in WFD of {'u': 0.07, 'g...",bluer/bluer_indx0_v2.0_10yrs.db,http://epyc.astro.washington.edu/~lynnej/opsim...
bluer_indx1_v2.0_10yrs,[bluer],[2],More u and g - filter ratio in WFD of {'u': 0....,bluer/bluer_indx1_v2.0_10yrs.db,http://epyc.astro.washington.edu/~lynnej/opsim...
carina_v2.0_10yrs,[microsurveys],[2],One week per year observing Carina,microsurveys/virgo_cluster/virgo_cluster_v2.0_...,http://epyc.astro.washington.edu/~lynnej/opsim...
...,...,...,...,...,...
vary_nes_nesfrac0.50_v2.0_10yrs,[vary_nes],[2],NES at 50% WFD level,vary_nes/vary_nes_nesfrac0.50_v2.0_10yrs.db,http://epyc.astro.washington.edu/~lynnej/opsim...
vary_nes_nesfrac0.55_v2.0_10yrs,[vary_nes],[2],NES at 55% WFD level,vary_nes/vary_nes_nesfrac0.55_v2.0_10yrs.db,http://epyc.astro.washington.edu/~lynnej/opsim...
vary_nes_nesfrac0.75_v2.0_10yrs,[vary_nes],[2],NES at 75% WFD level,vary_nes/vary_nes_nesfrac0.75_v2.0_10yrs.db,http://epyc.astro.washington.edu/~lynnej/opsim...


By default, `get_runs` retrives the runs data from the same json file as `archive.get_family_descriptions`, and also has an argument to download the data from a different source.

`get_family_descriptions` and `get_runs` load the same data, but the former is indexed by families, with one row per family; and the later by runs, with one row per run. In the former case, values that vary by run for the same family have list values, while in the later case values that vary by family for the same run have list values.

If you want a `DataFrame` with one row per run/family combination, such that there are no columns with list-valued cells, you can `explode` the `pandas.DataFrame` returned by `get_runs` (or `get_family_descriptions`) and set the `family` column to be the index, or use the `get_family_runs` shorthand:

In [8]:
family_runs = archive.get_family_runs()
family_runs

,run,brief,filepath,version,reference,description,url
family,,,,,,,
baseline,baseline_v2.0_10yrs,v2.0 baseline,baseline/baseline_v2.0_10yrs.db,2,baseline_v2.0_10yrs,**baseline** : The v2.0 baseline simulation. T...,http://epyc.astro.washington.edu/~lynnej/opsim...
retro,baseline_retrofoot_v2.0_10yrs,v1.x footprint but new code capabilities,retro/baseline_retrofoot_v2.0_10yrs.db,2,baseline_v2.0_10yrs,**retro** : The retro family provides two simu...,http://epyc.astro.washington.edu/~lynnej/opsim...
retro,retro_baseline_v2.0_10yrs,v1.x baseline with rubin_sim (without new code...,retro/retro_baseline_v2.0_10yrs.db,2,baseline_v2.0_10yrs,**retro** : The retro family provides two simu...,http://epyc.astro.washington.edu/~lynnej/opsim...
rolling,rolling_ns2_rw0.5_v2.0_10yrs,Rolling half-sky (2 regions) 50%,rolling/rolling_ns2_rw0.5_v2.0_10yrs.db,2,baseline_v2.0_10yrs,**rolling** : The updated baseline contains a ...,http://epyc.astro.washington.edu/~lynnej/opsim...
rolling,rolling_ns3_rw0.5_v2.0_10yrs,Rolling third-sky (3 regions) 50%,rolling/rolling_ns2_rw0.9_v2.0_10yrs.db,2,baseline_v2.0_10yrs,**rolling** : The updated baseline contains a ...,http://epyc.astro.washington.edu/~lynnej/opsim...
...,...,...,...,...,...,...,...
microsurveys,twilight_neo_nightpattern3v2.0_10yrs,Twilight neo observations 1 night on 2 off,microsurveys/local_gals/local_gal_bindx0_v2.0_...,2,baseline_v2.0_10yrs,**microsurveys** : There are special programs ...,http://epyc.astro.washington.edu/~lynnej/opsim...
microsurveys,twilight_neo_nightpattern6v2.0_10yrs,Twilight neo observations 3 night on 4 off,microsurveys/carina/carina_v2.0_10yrs.db,2,baseline_v2.0_10yrs,**microsurveys** : There are special programs ...,http://epyc.astro.washington.edu/~lynnej/opsim...
microsurveys,twilight_neo_nightpattern5v2.0_10yrs,Twilight neo observations 4 night on 4 off,microsurveys/short_exp/short_exp_v2.0_10yrs.db,2,baseline_v2.0_10yrs,**microsurveys** : There are special programs ...,http://epyc.astro.washington.edu/~lynnej/opsim...


This makes it easy to reference just the runs from a family (or set of families) of interest.

`get_family_runs` reads the run metadata from the same source as `get_runs`, and (like `get_runs`) alternate sources can be specified by an argument.

First, let's look at a list of all families, and how many runs are in each:

In [9]:
family_runs.groupby("family").agg({"run": "count"})

,run
family,
baseline,1
bluer,2
ddf,2
long_gaps,16
long_gaps_nopair,16
long_u,2
microsurveys,19
noroll,1
presto,12


If I want to work with just runs in `baseline` or `rolling`, I can build `pandas.DataFrame` of such runs by slicing `family_runs`:

In [10]:
my_runs = family_runs.loc[["baseline", "rolling"]]
my_runs

,run,brief,filepath,version,reference,description,url
family,,,,,,,
baseline,baseline_v2.0_10yrs,v2.0 baseline,baseline/baseline_v2.0_10yrs.db,2,baseline_v2.0_10yrs,**baseline** : The v2.0 baseline simulation. T...,http://epyc.astro.washington.edu/~lynnej/opsim...
rolling,rolling_ns2_rw0.5_v2.0_10yrs,Rolling half-sky (2 regions) 50%,rolling/rolling_ns2_rw0.5_v2.0_10yrs.db,2,baseline_v2.0_10yrs,**rolling** : The updated baseline contains a ...,http://epyc.astro.washington.edu/~lynnej/opsim...
rolling,rolling_ns3_rw0.5_v2.0_10yrs,Rolling third-sky (3 regions) 50%,rolling/rolling_ns2_rw0.9_v2.0_10yrs.db,2,baseline_v2.0_10yrs,**rolling** : The updated baseline contains a ...,http://epyc.astro.washington.edu/~lynnej/opsim...
rolling,rolling_ns2_rw0.9_v2.0_10yrs,Rolling half-sky (2 regions) 90%,rolling/rolling_ns3_rw0.5_v2.0_10yrs.db,2,baseline_v2.0_10yrs,**rolling** : The updated baseline contains a ...,http://epyc.astro.washington.edu/~lynnej/opsim...
rolling,rolling_ns3_rw0.9_v2.0_10yrs,Rolling third-sky (3 regions) 90%,rolling/rolling_ns3_rw0.9_v2.0_10yrs.db,2,baseline_v2.0_10yrs,**rolling** : The updated baseline contains a ...,http://epyc.astro.washington.edu/~lynnej/opsim...


## Getting summary metrics on runs

`get_metric_summaries` will retrieve the "summary" results of MAF for these runs into a `pandas.DataFrame`. It can either be called with no arguments (in which case it will get all summary metrics for all runs), or you can supply argumets to specificy which runs, run familes, and metrics you want. Be default, summary data is downloaded from the URL specified by `archive.DEFAULT_SUMMARY_SOURCE`. Users may pass an alternate source (URL or file name) to `get_metric_summaries` to load the data from elsewhere.

In [11]:
archive.SUMMARY_SOURCE

'https://raw.githubusercontent.com/lsst-pst/survey_strategy/main/fbs_2.0/summary_11_8.csv'

For example, to get only metrics on runs in the `version_baselines` family:

In [12]:
archive.get_metric_summaries(run_families="rolling")

OpsimRun,rolling_ns2_rw0.5_v2.0_10yrs,rolling_ns2_rw0.9_v2.0_10yrs,rolling_ns3_rw0.5_v2.0_10yrs,rolling_ns3_rw0.9_v2.0_10yrs
metric,,,,
fOArea fO All visits HealpixSlicer,12614.583488,12893.228978,12729.566717,13255.803832
fOArea/benchmark fO All visits HealpixSlicer,0.700810,0.716290,0.707198,0.736434
fONv MedianNvis fO All visits HealpixSlicer,838.000000,839.000000,838.000000,841.000000
fONv MinNvis fO All visits HealpixSlicer,787.000000,787.000000,788.000000,788.000000
fONv/benchmark MedianNvis fO All visits HealpixSlicer,1.015758,1.016970,1.015758,1.019394
...,...,...,...,...
N(-3Sigma) AGN SF_error z band HealpixSlicer,0.000000,0.000000,0.000000,0.000000
AGN SF_error z band HealpixSlicer,29541.000000,29529.000000,29601.000000,29521.000000
25th%ile AGN SF_error z band HealpixSlicer,0.036877,0.037055,0.037091,0.037427


It returned more than 37000 metrics! This is more than necessary or convenient for most purposes.

## Metric sets

Rather than sort through all these metrics, you can work with pre-defined sets of metrics generated for a variety of purposes. `get_metric_sets` loads definitions of sets of metrics (and other metric metadata) from a URL specified by `archive.DEFAULT_METRIC_SET_SOURCE`. Users can load this data by passing the URL or file name as the argument to `get_metric_sets`. *Metric set definitions cannot be arbitrarily mixed with summary sources: each version of summary source must be matched with a corresponding metric set source.*

In [13]:
archive.METRIC_SET_SOURCE

'https://raw.githubusercontent.com/lsst-pst/survey_strategy/main/fbs_2.0/metric_sets.json'

You can get the set definitions using `get_metric_sets`:

In [14]:
metric_sets = archive.get_metric_sets()
metric_sets

HTTPError: HTTP Error 404: Not Found

Slicing this `pandas.DataFrame` will give you the metrics for just the sets you specify:

In [ ]:
metric_sets.loc["srd"]

In [ ]:
metric_sets.loc["Nvis"]

In [ ]:
metric_sets.loc[["srd", "radar"], :]

You can get a list of available sets by listing the values of the index:

In [ ]:
archive.get_metric_sets().groupby("metric set").first().index

You can get metric summaries by getting the list of metrics from the `DataFrame` through the `metrics` option, or you can just set the `metric_sets` option directly:

In [ ]:
archive.get_metric_summaries(run_families="rolling", metric_sets="radar")

## Normalizing summary metrics

When comparing many runs with many metrics, it helps if each metric behaves similarly. As recorded, though, the numeric values of different metrics mean different things. For example, some metrics are better when they have higher values, others are better with lower values. Furthermore, they are all scaled differently.

For ease of comparison, we can transform all of them such that they take a value of 1 if they are equally good to some baseline, positive if better, negative if worse.

This comparison continues to be rough: different metrics continue to have different units, and so are not directly comparable. Still, it is a rough improvement.

Pick a family of runs and a set of metrics to use in this example:

In [ ]:
this_family = ["baseline", "rolling"]
this_metric_set = "srd"

We need to pick a reference run to define to have a normalized value of 1:

In [ ]:
baseline_run = "baseline_v2.0_10yrs"

Now get all the (unnormalized) metrics summary values:

In [ ]:
summary = archive.get_metric_summaries(this_family, this_metric_set)

Normalize it and look at the results:

In [ ]:
norm_summary = archive.normalize_metric_summaries(baseline_run, summary)
norm_summary

## Cartesian plots

This would be easier to interpret if presented graphically, for which you can use `plot_run_metric`:

In [ ]:
archive.plot_run_metric(
    summary,
    baseline_run=baseline_run,
)

The labels on this plot are long and hard to interpret, but shorter names for both runs and metrics are available in the `pandas.DataFrames` we have already downloaded, and we can build transformations from these `DataFrame`s.

In [ ]:
metric_label_map = metric_sets.loc[this_metric_set, "short_name"]
metric_label_map

In [ ]:
run_label_map = family_runs.loc[this_family, ["run", "brief"]].set_index("run")["brief"]
run_label_map

These can be passed to `plot_metric_summary` to replace the labels:

In [ ]:
archive.plot_run_metric(
    summary,
    baseline_run=baseline_run,
    run_label_map=run_label_map,
    metric_label_map=metric_label_map,
)

The `vertical_quantity` and `horizontal_quantity` options will let you set which axis (horizontal, vertical, or color) is mapped to which quantity (run, metric name, metric value), and additional arguments set the color maps, markers, and line styles connecting the points:

In [ ]:
import matplotlib as mpl

this_metric_set = "radar"

this_family = ["baseline", "microsurveys"]
run_label_map = family_runs.loc[this_family, ["run", "brief"]].set_index("run")["brief"]
metric_label_map = metric_sets.loc[this_metric_set, "short_name"]

summary = archive.get_metric_summaries(this_family, this_metric_set)
metric_label_map = metric_sets.loc[this_metric_set, "short_name"]
archive.plot_run_metric(
    summary,
    baseline_run=baseline_run,
    vertical_quantity="value",
    horizontal_quantity="run",
    run_label_map=run_label_map,
    metric_label_map=metric_label_map,
    cmap=mpl.cm.tab10,
    linestyles=["-", ":", "--", "-."],
    markers=["o", "v", "^", "<", ">", "*", "H", "D"],
)

## Mesh plots

Alternately, you can color code the metric value itself using `plot_run_metric_mesh`:

In [ ]:
this_metric_set = ["Nvis", "radar"]
metric_label_map = metric_sets.loc[this_metric_set, "short_name"].droplevel(
    "metric set"
)

this_family = ["baseline", "microsurveys"]
run_label_map = family_runs.loc[this_family, ["run", "brief"]].set_index("run")["brief"]

summary = archive.get_metric_summaries(this_family, this_metric_set)

archive.plot_run_metric_mesh(
    summary,
    baseline_run=baseline_run,
    run_label_map=run_label_map,
    metric_label_map=metric_label_map,
)

## Radar plots

<span style='color:red'>TODO: to get this to work, I had to add `from .radarPlot import radar` to `rubin_sim/maf/runComparison/__init__.py`.</span>

Finally, if the numbers of runs and metrics are manageable, you can compare different metrics of different runs with a radar plot.

Let's select a modest collection of metrics and family of runs, and build a summary:

In [ ]:
family_runs.loc[["baseline", "retro", "rolling"], :]

In [ ]:
this_metric_set = "radar"
this_family = ["baseline", "retro", "rolling"]
summary = archive.get_metric_summaries(this_family, this_metric_set)

The radar plot function requires that the data already be normalized, so normalize it:

In [ ]:
norm_summary = archive.normalize_metric_summaries(baseline_run, summary)

The radar plot function takes the run and metric names from the `DataFrame` row and column names, so we can use short name by renaming the rows and columns:

In [ ]:
metric_label_map = metric_sets.loc[this_metric_set, "short_name"]
run_label_map = family_runs.loc[this_family, ["run", "brief"]].set_index("run")["brief"]
norm_summary.rename(columns=run_label_map, index=metric_label_map, inplace=True)

Make the radar plot:

In [ ]:
maf.radar(norm_summary.T, bbox_to_anchor=(3.7, 0))

## Plotting yet more metrics and runs

Multiple sets of metrics and families of runs can be retrieved and plotted at once, and these can be supplemented by additional individual runs and metrics:

In [ ]:
these_metric_sets = ["srd", "Nvis", "radar"]
these_families = ["baseline", "retro", "rolling"]
extra_runs = ["ddf_frac_ddf_per0.6_v2.0_10yrs", "ddf_frac_ddf_per1.6_v2.0_10yrs"]
extra_metrics = [
    "Effective Area (deg) ExgalM5_with_cuts i band non-DD year 10 HealpixSlicer",
]
summary = archive.get_metric_summaries(
    these_families, these_metric_sets, runs=extra_runs, metrics=extra_metrics
)

Because the slicing in pandas will return a multilevel index when multiple runs or families are sliced on, a little additional processing is needed to get a mapping from run or index name alone:

In [ ]:
these_metrics = list(summary.index)
metric_label_map = (
    metric_sets.loc[(slice(None), these_metrics), "short_name"]
    .groupby("metric")
    .first()
)
these_runs = list(summary.columns)
run_label_map = (
    family_runs[["run", "brief"]]
    .set_index("run")
    .loc[these_runs, "brief"]
    .groupby("run")
    .first()
)

In [ ]:
archive.plot_run_metric_mesh(
    summary,
    baseline_run=baseline_run,
    run_label_map=run_label_map,
    metric_label_map=metric_label_map,
)

## Plotting other metrics

Not all metrics present in the summary table have corresponding columns in the `metrics_set` data. If they do not, they cannot be normalized, and do not show up in the plots of normalized values.
For exmample, minimum WFD depth values have no normalization values:

In [ ]:
min_depth_metrics = tuple(f"Min CoaddM5 WFD {b} band HealpixSlicer" for b in "ugrizy")
min_depth_metrics

In [ ]:
metric_sets.loc[(slice(None), min_depth_metrics), :]

We can still load the un-normalied metric values:

In [ ]:
summary = archive.get_metric_summaries(["baseline", "retro"], metrics=min_depth_metrics)
summary

In [ ]:
archive.plot_run_metric(summary)

If you want to properly normalize them, you can create you own metrics `DataFrame`:

In [ ]:
import pandas as pd

my_metrics = (
    pd.DataFrame(
        {
            "metric set": "Min WFD depth",
            "metric": min_depth_metrics,
            "short_name": [f"Min {b} band depth in WFD" for b in "ugrizy"],
            "style": "-",
            "invert": False,
            "mag": True,
        }
    )
    .set_index("metric set")
    .set_index("metric", append=True, drop=False)
)
my_metrics

In [ ]:
archive.plot_run_metric(
    summary,
    baseline_run=baseline_run,
    run_label_map=run_label_map,
    metric_label_map=my_metrics.droplevel("metric set")["short_name"],
    metric_set_source=my_metrics,
)

## Running additional MAF metrics

If the summary metrics are inadequate for what you need, you can download the opsim databases using URLs found in the `runs` `DataFrame` we downloaded above (using `get_runs`):

In [ ]:
runs

You will need to specify a directory into which to download runs. To set where this notebook stores the run databases it downloads, uncomment the following cell and set the `out_dir` to your desired location:

In [ ]:
# out_dir = '.'

For the purposes of this example, if you do not set the `out_dir`, this notebook creates a temporare directory (which will be deleted when the notebook process ends):

In [ ]:
from tempfile import TemporaryDirectory
import os

try:
    print(f"Output directory for downloaded opsim run databases is set to {out_dir}.")
except NameError:
    temp_dir = TemporaryDirectory(dir=os.getcwd(), prefix="tmp_opsim_runs_")
    out_dir = temp_dir.name
    print(
        f"Temporary directory {out_dir} created for the storage of opsim databases downloaded by this notebook."
    )
    print("These will be deleted when this notebook is stopped or restarted!")

You also need to build a list of runs to download. One easy way is to select the runs that are part of a family, as present in the `family_runs` `DataFrame` we created with `get_family_runs` above:

In [ ]:
my_runs = family_runs.loc[["baseline", "retro"], "run"].tolist()
my_runs

Now you can download the opsim output of interest into your runs directory:

In [ ]:
dest_fnames = archive.download_runs(my_runs, out_dir)
dest_fnames

Note that the return is a `pandas.Series` that maps run name to the file name it was downloaded into. You can now proceed to use these opsim database for your own MAF analysis.